<a href="https://colab.research.google.com/github/vitor-yuichi/covid_analysis_project/blob/main/codes/COVID_analysis_complete.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project 1: COVID-19 Spread Analysis with Python
Project Idea – The objective of this project is to implement a real-time dashboard for COVID 19 spread analysis.

This live dashboard will provide many insights for the study of coronavirus spread.


In [ ]:
import folium
import flask
import pandas as pd 
import matplotlib.pyplot as plt 
from google.colab import drive 
drive.mount('/content/gdrive')
from google.colab import files


Mounted at /content/gdrive


# Importing the CSV file (Covid data)
1. Load the dataset and collect the top 15 regions having the largest corona cases:

In [ ]:
df=pd.read_csv(r'/content/gdrive/My Drive/Colab Notebooks/project/covid19.csv')
pd.set_option('display.max_rows', 500)
df


,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key
0,45001.0,Abbeville,South Carolina,US,2020-04-03 22:46:37,34.223334,-82.461707,6,0,0,0,"Abbeville, South Carolina, US"
1,22001.0,Acadia,Louisiana,US,2020-04-03 22:46:37,30.295065,-92.414197,72,1,0,0,"Acadia, Louisiana, US"
2,51001.0,Accomack,Virginia,US,2020-04-03 22:46:37,37.767072,-75.632346,8,0,0,0,"Accomack, Virginia, US"
3,16001.0,Ada,Idaho,US,2020-04-03 22:46:37,43.452658,-116.241552,307,3,0,0,"Ada, Idaho, US"
4,19001.0,Adair,Iowa,US,2020-04-03 22:46:37,41.330756,-94.471059,1,0,0,0,"Adair, Iowa, US"
...,...,...,...,...,...,...,...,...,...,...,...,...
2619,NaN,NaN,NaN,Vietnam,2020-04-03 22:46:20,14.058324,108.277199,237,0,85,152,Vietnam
2620,NaN,NaN,NaN,West Bank and Gaza,2020-04-03 22:46:20,31.952200,35.233200,194,1,21,172,West Bank and Gaza
2621,NaN,NaN,NaN,Zambia,2020-04-03 22:46:20,-13.133897,27.849332,39,1,2,36,Zambia
2622,NaN,NaN,NaN,Zimbabwe,2020-04-03 22:46:20,-19.015438,29.154857,9,1,0,8,Zimbabwe


In [ ]:
regions_cases=df.groupby(['Country_Region']).sum().sort_values(by='Confirmed', ascending=False).reset_index()
regions_cases15=regions_cases.loc[0:14]
regions_cases15=regions_cases15.drop(columns=['FIPS', 'Lat', 'Long_']) #codigo alternativo para essa parte regions_cases.nlargest(15, 'Confirmed')[['Confirmed']]


#Creating a function for the first 15 regions 

In [ ]:
def regions_15 (n=15):
  df=pd.read_csv(r'/content/gdrive/My Drive/Colab Notebooks/project/covid19.csv')
  regions_cases=df.groupby(['Country_Region']).sum().sort_values(by='Confirmed', ascending=False).reset_index()
  regions_cases_function=regions_cases.loc[0:n-1]
  regions_cases_function=regions_cases_function.drop(columns=['FIPS', 'Lat', 'Long_'])
  return regions_cases_function

# Creating a sample map 
2. Make a sample map using the folium package and write a function to make circles on active corona cases regions
I will change the icons

In [ ]:
df

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key
0,45001.0,Abbeville,South Carolina,US,2020-04-03 22:46:37,34.223334,-82.461707,6,0,0,0,"Abbeville, South Carolina, US"
1,22001.0,Acadia,Louisiana,US,2020-04-03 22:46:37,30.295065,-92.414197,72,1,0,0,"Acadia, Louisiana, US"
2,51001.0,Accomack,Virginia,US,2020-04-03 22:46:37,37.767072,-75.632346,8,0,0,0,"Accomack, Virginia, US"
3,16001.0,Ada,Idaho,US,2020-04-03 22:46:37,43.452658,-116.241552,307,3,0,0,"Ada, Idaho, US"
4,19001.0,Adair,Iowa,US,2020-04-03 22:46:37,41.330756,-94.471059,1,0,0,0,"Adair, Iowa, US"
...,...,...,...,...,...,...,...,...,...,...,...,...
2619,NaN,NaN,NaN,Vietnam,2020-04-03 22:46:20,14.058324,108.277199,237,0,85,152,Vietnam
2620,NaN,NaN,NaN,West Bank and Gaza,2020-04-03 22:46:20,31.952200,35.233200,194,1,21,172,West Bank and Gaza
2621,NaN,NaN,NaN,Zambia,2020-04-03 22:46:20,-13.133897,27.849332,39,1,2,36,Zambia
2622,NaN,NaN,NaN,Zimbabwe,2020-04-03 22:46:20,-19.015438,29.154857,9,1,0,8,Zimbabwe


In [ ]:
df=df.dropna() #remove all blank values (NaN)
m=folium.Map(location=[34.223334, -82.461707],
           tiles='Stamen Toner', zoom_start=5)
icon_url='https://cdn.pixabay.com/photo/2020/04/18/08/37/coronavirus-5058258_960_720.png'
icons = folium.features.CustomIcon(icon_url,icon_size=(28, 30))
df.apply(lambda row: folium.Marker(location=[row['Lat'], row['Long_']], 
popup=row['Combined_Key'], icon=folium.features.CustomIcon(icon_url,icon_size=(10, 10))).add_to(m), axis=1 )
html_map=m._repr_html_()

#Flask setting 
3. Now do the required settings for flask app

In [ ]:
!pip install flask-ngrok

In [ ]:
region_15_zipped=regions_cases15[['Country_Region', 'Confirmed']]

In [ ]:

pairs=[(country,confirmed) for country,confirmed in zip(region_15_zipped['Country_Region'],region_15_zipped['Confirmed'])]


In [ ]:
static_path='/content/gdrive/MyDrive/Colab Notebooks/project/templates/static'
from flask import Flask,render_template
from flask_ngrok import run_with_ngrok
app=Flask('covid_analysis_app', template_folder='/content/gdrive/My Drive/Colab Notebooks/project/templates', 
          static_folder=static_path)

run_with_ngrok(app)  
@app.route('/')
def home():
  return render_template("home.html", table=region_15_zipped, cmap=html_map, pairs=pairs)
if 'covid_analysis_app'=="__main__":
    app.run(debug=True)

In [ ]:
app.run()


 * Serving Flask app "covid_analysis_app" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://25db2ef1a7c4.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [27/Jul/2021 03:25:11] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Jul/2021 03:25:12] "GET /static/sty.css HTTP/1.1" 200 -
127.0.0.1 - - [27/Jul/2021 03:25:26] "GET /favicon.ico HTTP/1.1" 404 -
